<a href="https://colab.research.google.com/github/Pavitra-khare/DA6401_ASS3_withAtten/blob/main/ASS3_WithAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and login

In [ ]:
!pip install pytorch_lightning
!pip install wandb


In [ ]:
import wandb
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import warnings
import torch
import torch.nn.functional as F
torch.cuda.empty_cache()
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch.utils.data as data
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as patches
warnings.filterwarnings("ignore")
#function to create 3*3 grid of heatMap
import matplotlib.font_manager as fm


In [ ]:
wandb.login(key="7ff4b917f4a107c44e50219b5fd09ece84a34600")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# Data Preprocessing

In [ ]:
def read_file_0(filepath):
    def process_row(text_row):
        return list(text_row)

    def collect_characters(reader_obj):
        all_chars = []
        for entry in reader_obj:
            all_chars += process_row(entry[0])
        return all_chars

    with open(filepath, mode='r') as file_handle:
        csv_reader = csv.reader(file_handle,delimiter='\t')
        return collect_characters(csv_reader)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

'''Location of your CSV file (Extracted file)
Location of your CSV file if on kaggle than zip file is fine'''

trainFilepath="/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
valFilePath="/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
testFilePath="/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"

In [ ]:


chars = read_file_0(trainFilepath)
setChar=set(chars)
setChar.add('|')
setOfchar = list(setChar)

# Create the association between characters and their corresponding integer indices
char_to_idx_latin= {char: i+1 for i, char in enumerate(setOfchar)}

In [ ]:
def read_file_1(trainFilepath):
    def extract_chars(text):
        return list(text)

    def accumulate(reader_obj):
        buffer = []
        for entry in reader_obj:
            buffer += extract_chars(entry[1])
        return buffer

    with open(trainFilepath, 'r') as file_stream:
        csv_rows = csv.reader(file_stream,delimiter='\t')
        return accumulate(csv_rows)

In [ ]:
maxLenDev=0

chars = read_file_1(trainFilepath)
setChar=set(chars)
setChar.add('|')
setOfchar = list(setChar)

charToIndLang ={char: i+1 for i, char in enumerate(setOfchar)}

In [ ]:
with open(trainFilepath, 'r') as csv_file:
    reader = csv.reader(csv_file,delimiter='\t')
    max_length = 0

    def update_max(current_max, candidate):
        return candidate if candidate > current_max else current_max

    for record in reader:
        length = len(record[0])
        max_length = update_max(max_length, length)

    maxLenEng = max_length

In [ ]:
with open(trainFilepath, 'r') as csv_stream:
    reader_obj = csv.reader(csv_stream,delimiter='\t')
    longest = 0

    def get_larger(a, b):
        return b if b > a else a

    for line in reader_obj:
        current_length = len(line[1])
        longest = get_larger(longest, current_length)

    maxLenDev = longest

characters in words -> indices












In [ ]:
def convert_characters_to_indices(word, dictionary):
    def safe_lookup(char):
        return dictionary[char] if char in dictionary else -1

    mapped = [safe_lookup(ch) for ch in word]
    filtered = list(filter(lambda idx: idx >= 0, mapped))
    return filtered

In [ ]:
def adjust_sequence_length(indices, maximumLength):
    current_length = len(indices)

    def trim(seq, length):
        return seq[:length]

    def pad(seq, total_length):
        padding_needed = total_length - len(seq)
        return seq + [0] * padding_needed

    if current_length > maximumLength:
        return trim(indices, maximumLength)
    elif current_length < maximumLength:
        return pad(indices, maximumLength)
    return indices


In [ ]:
def convert_indices_to_tensor(indices, dictionary):
    token = dictionary.get('|', 0)

    def add_delimiters(seq, token_id):
        return [token_id] + seq + [token_id]

    sequence = add_delimiters(indices, token)
    tensor_obj = torch.tensor(sequence, device=device)
    return tensor_obj

In [ ]:
def convert_word_to_indices(word, maximumLength, dict):
    char_ids = convert_characters_to_indices(word, dict)

    def standardize_length(seq, target_len):
        return adjust_sequence_length(seq, target_len)

    resized = standardize_length(char_ids, maximumLength)
    tensor_out = convert_indices_to_tensor(resized, dict)
    return tensor_out


In [ ]:
def assign_tensor_to_generated_sequences(sequence):
    tokens = sequence.split()

    def concatenate(tokens):
        return ''.join(tokens)

    def compute_total_length(tokens):
        return sum(len(token) for token in tokens)

    combined = concatenate(tokens)
    total_len = compute_total_length(tokens)

    return combined, total_len

In [ ]:
def assemble_tensor(final_tensor, partition_size=1):
    def safe_partition_size(size):
        return max(1, size)

    chunk_size = safe_partition_size(partition_size)
    segments = [final_tensor[idx:idx + chunk_size] for idx in range(0, len(final_tensor), chunk_size)]

    return segments


In [ ]:
def assemble_assigned_generated_seq(path):
    combined_seq, total_chars = assign_tensor_to_generated_sequences(path)

    def determine_chunk_size(length, divisor=4):
        return max(1, length // divisor)

    segment_length = determine_chunk_size(total_chars)
    partitioned = assemble_tensor(combined_seq, segment_length)

    return partitioned


In [ ]:
def generate_indices(row):
    src_text = row[0]
    tgt_text = row[1]

    def build_indexed_tensor(text, max_len, char_map):
        return convert_word_to_indices(text, max_len, char_map)

    source_tensor = build_indexed_tensor(src_text, maxLenEng, char_to_idx_latin)
    target_tensor = build_indexed_tensor(tgt_text, maxLenDev, charToIndLang)

    return source_tensor, target_tensor


In [ ]:
pairs_v = []

with open(valFilePath, 'r') as val_file:
    csv_reader = csv.reader(val_file,delimiter='\t')

    def process_and_append(pairs, record):
        eng_tensor, hin_tensor = generate_indices(record)
        pairs.append([eng_tensor, hin_tensor])

    for entry in csv_reader:
        process_and_append(pairs_v, entry)


In [ ]:
pairs_t = []

with open(testFilePath, 'r') as test_file:
    test_reader = csv.reader(test_file,delimiter='\t')

    def append_indexed_pair(container, data_row):
        src_tensor, tgt_tensor = generate_indices(data_row)
        container.append([src_tensor, tgt_tensor])

    for record in test_reader:
        append_indexed_pair(pairs_t, record)


In [ ]:
pairs = []

with open(trainFilepath, 'r') as train_file:
    train_reader = csv.reader(train_file,delimiter='\t')

    def process_row_and_store(storage, row_data):
        input_tensor, target_tensor = generate_indices(row_data)
        storage.append([input_tensor, target_tensor])

    for entry in train_reader:
        process_row_and_store(pairs, entry)


#DATA LOADER

In [ ]:
batch_size = 32
train_shuffle = True
eval_shuffle = False

def create_loader(dataset, size, shuffle_flag):
    return torch.utils.data.DataLoader(dataset, batch_size=size, shuffle=shuffle_flag)

dataloaderTrain = create_loader(pairs, batch_size, train_shuffle)
dataloaderVal = create_loader(pairs_v, batch_size, eval_shuffle)
dataloaderTest = create_loader(pairs_t, batch_size, eval_shuffle)


#CLASS FOR ENCODER DECODER and seq2seq

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttnEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embed_dim, rnn_type, dropout, num_layers, is_bidirectional):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.is_bidirectional = is_bidirectional
        self.embedding = nn.Embedding(input_dim, embed_dim)

        rnn_map = {
            "GRU": nn.GRU,
            "LSTM": nn.LSTM,
            "RNN": nn.RNN
        }
        self.rnn = rnn_map[rnn_type](embed_dim, hidden_dim, dropout=dropout, num_layers=num_layers, bidirectional=is_bidirectional)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        return output, hidden


class AttnDecoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embed_dim, rnn_type, dropout, num_layers, is_bidirectional, max_seq_len):
        super().__init__()
        self.max_seq_len = max_seq_len + 2
        self.hidden_dim = hidden_dim
        self.embed_dim = embed_dim
        self.is_bidirectional = is_bidirectional
        self.rnn_type = rnn_type

        self.embedding = nn.Embedding(output_dim, embed_dim)

        self.attn = nn.Linear(hidden_dim + embed_dim, self.max_seq_len)

        concat_dim = hidden_dim * (2 if is_bidirectional else 1) + embed_dim
        self.attn_combine = nn.Linear(concat_dim, hidden_dim)

        rnn_input_dim = hidden_dim
        rnn_cls = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[rnn_type]
        self.rnn = rnn_cls(rnn_input_dim, hidden_dim, dropout=dropout, num_layers=num_layers, bidirectional=is_bidirectional)

        output_proj_dim = hidden_dim * (2 if is_bidirectional else 1)
        self.output_layer = nn.Linear(output_proj_dim, output_dim)

    def forward(self, input_token, hidden_state, encoder_outputs):
        embedded = self.embedding(input_token.unsqueeze(0))  # shape: (1, batch, embed_dim)

        if self.rnn_type == "LSTM":
            hidden_tensor = hidden_state[0][0]
        else:
            hidden_tensor = hidden_state[0]

        attention_input = torch.cat((embedded[0], hidden_tensor), dim=1)
        attn_weights = F.softmax(self.attn(attention_input), dim=1)

        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs.permute(1, 0, 2)).squeeze(1)
        combined = torch.cat((embedded[0], context), dim=1)
        rnn_input = F.relu(self.attn_combine(combined)).unsqueeze(0)

        if self.rnn_type == "LSTM":
            output, hidden = self.rnn(rnn_input, hidden_state)
        else:
            output, hidden = self.rnn(rnn_input, hidden_state)

        logits = self.output_layer(output.squeeze(0))
        return logits, hidden, attn_weights




# CLASS FOR SEQ2SEQ

In [ ]:
#Sequence class for with attention
class Seq2SeqAttn(pl.LightningModule):
    def __init__(self, input_dim, output_dim, hidden_dim, embed_dim, rnn_type, dropout, enc_layers, dec_layers, is_bidir, lr, max_src_len):
        super().__init__()

        self.lr = lr
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.rnn_type = rnn_type
        self.is_bidir = is_bidir
        self.mult = 2 if is_bidir else 1

        self.encoder = AttnEncoder(input_dim, hidden_dim, embed_dim, rnn_type, dropout, enc_layers, is_bidir)
        self.decoder = AttnDecoder(output_dim, hidden_dim, embed_dim, rnn_type, dropout, dec_layers, is_bidir, max_src_len)

        self.trainLoss, self.trainAccuracy = [], []
        self.valLoss, self.valAccuracy = [], []
        self.test_loss, self.testAccuracy = [], []

        self.attentionWeights = []
        self.counter = 0

    def forward(self, src_seq, tgt_seq, teacher_ratio=0.5):
        batch_size, max_len = tgt_seq.shape
        src_seq = src_seq.transpose(0, 1)
        attn_map = torch.zeros(max_len, batch_size, self.decoder.max_seq_len).to(self.device)

        vocab_size = self.decoder.output_layer.out_features
        encoder_out, hidden = self.encoder(src_seq)
        predictions = torch.zeros(max_len, batch_size, vocab_size).to(self.device)

        token = tgt_seq[:, 0]
        for t in range(1, max_len):
            pred, hidden, attn_map[t] = self.decoder(token, hidden, encoder_out)
            predictions[t] = pred
            token = pred.argmax(1) if teacher_ratio < torch.rand(1).item() else tgt_seq[:, t]

        return predictions, attn_map

    def get_model_output(self, src, tgt, teacher_ratio=0.5):
        preds, _ = self(src, tgt, teacher_ratio)
        return preds.permute(1, 0, 2)

    def prepare_expected_tensor(self, output, target):
        expected = torch.zeros_like(output)
        expected[torch.arange(output.shape[0]), torch.arange(output.shape[1]).unsqueeze(1), target.cpu()] = 1
        return expected

    def prepare_output_expected_tensors(self, output, target):
        expected = self.prepare_expected_tensor(output, target)
        dim = output.shape[-1]
        return output[1:].view(-1, dim), expected[1:].view(-1, dim), target[1:].view(-1)

    def calculate_loss(self, preds, expected):
        return self.loss_fn(preds.to(device), expected.to(device))

    def calculate_accuracy(self, preds, target):
        return self.accuracy(preds, target)

    def append_metrics(self, loss_val, acc_val):
        self.trainLoss.append(torch.tensor(loss_val))
        self.trainAccuracy.append(torch.tensor(acc_val))

    def training_step(self, batch, batch_idx):
        src, tgt = batch

        # 1. Forward pass  (seq_len, batch, vocab)
        seq_logits, _ = self(src, tgt, teacher_ratio=0.5)

        # 2. Accuracy wants batch-first
        batch_logits = seq_logits.permute(1, 0, 2)         # (batch, seq, vocab)
        acc = self.accuracy(batch_logits, tgt)

        # 3. Loss helper works with seq-first
        flat_logits, expected, _ = self.prepare_output_expected_tensors(seq_logits, tgt)
        loss = self.loss_fn(flat_logits, expected)

        # 4. Log
        self.append_metrics(loss, acc)
        return {"loss": loss}


    def get_output(self, src, tgt):
        raw_out, _ = self(src, tgt, 0.0)
        acc_out, _ = self(src, tgt, 0.0)
        return raw_out, acc_out.permute(1, 0, 2)

    def validation_step(self, batch, batch_idx):
        src, tgt = batch
        raw_out, acc_out = self.get_output(src, tgt)
        logits, expected, labels = self.prepare_output_expected_tensors(raw_out, tgt)
        val_loss = self.calculate_loss(logits, expected)
        val_acc = self.calculate_accuracy(acc_out, tgt)
        assemble_assigned_generated_seq('/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv')
        self.valLoss.append(torch.tensor(val_loss))
        self.valAccuracy.append(torch.tensor(val_acc))
        return {'loss': val_loss}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def loss_fn(self, output, target):
        return nn.CrossEntropyLoss()(output, target).mean()

    def accuracy(self, output, target):
        pred = output.argmax(dim=-1)
        matches = sum(torch.equal(target[i, 1:-1], pred[i, 1:-1]) for i in range(target.size(0)))
        return 100.0 * matches / target.size(0)

    def grid(self, src, output, target):
        pred = output.argmax(dim=-1)
        return [src[i, 1:-1] for i in range(target.size(0))], [target[i, 1:-1] for i in range(target.size(0))], [pred[i, 1:-1] for i in range(target.size(0))]

    def on_train_epoch_end(self):
        train_loss = torch.stack(self.trainLoss).mean()
        val_loss = torch.stack(self.valLoss).mean()
        train_acc = torch.stack(self.trainAccuracy).mean()
        val_acc = torch.stack(self.valAccuracy).mean()
        self.trainLoss.clear(), self.valLoss.clear(), self.trainAccuracy.clear(), self.valAccuracy.clear()
        print({"Train Loss": round(train_loss.item(), 3), "Train Accuracy": round(train_acc.item(), 3), "Validation Loss": round(val_loss.item(), 3), "Validation Accuracy": round(val_acc.item(), 3)})
        wandb.log({"Train Loss": train_loss, "Train Accuracy": train_acc, "Validation Loss": val_loss, "Validation Accuracy": val_acc})

    def get_out_attention(self, src, tgt):
        output, attn = self(src, tgt, 0.0)
        acc_out, attn_map = self(src, tgt, 0.0)
        return output, attn, acc_out.permute(1, 0, 2)

    def get_expected(self, output, target):
        mask = torch.zeros_like(output)
        mask[torch.arange(output.shape[0]), torch.arange(output.shape[1]).unsqueeze(1), target.cpu()] = 1
        return mask

    def test_step(self, batch, batch_idx):
        assemble_assigned_generated_seq('/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv')
        src, tgt = batch
        raw_out, attn_map, acc_out = self.get_out_attention(src, tgt)
        expected = self.get_expected(raw_out, tgt)

        dim = raw_out.shape[-1]
        logits = raw_out[1:].view(-1, dim)
        expected = expected[1:].view(-1, dim)
        labels = tgt[1:].view(-1)

        test_loss = self.loss_fn(logits.to(device), expected.to(device))
        test_acc = self.accuracy(acc_out, tgt)

        input_tokens, target_tokens, pred_tokens = self.grid(src, acc_out, tgt)
        assemble_assigned_generated_seq("string representations")

        target_texts, pred_texts, input_texts = [], [], []
        for seq in target_tokens:
            string = "".join([keyForVal(tok) for tok in seq])
            target_texts.append(string)
            assemble_assigned_generated_seq(string)

        for seq in pred_tokens:
            string = "".join([get_keyAttn(tok) for tok in seq])
            pred_texts.append(string)
            assemble_assigned_generated_seq(string)

        for seq in input_tokens:
            string = "".join([keyForInput(tok) for tok in seq])
            input_texts.append(string)
        assemble_assigned_generated_seq(string)

        self.testAccuracy.append(torch.tensor(test_acc))
        self.test_loss.append(torch.tensor(test_loss))
        # print({"for batch test_loss": test_loss, "testAccuracy": test_acc})
        # wandb.log({"Test Loss": test_loss, "Test Accuracy": test_acc})

        save_outputs_to_csv(input_texts, target_texts, pred_texts)
        # log_predictions_to_wandb(inputs, target_outputs, predicted_outputs)
        if(self.counter<1):
          s(input_texts, pred_texts, attn_map)
          self.counter=self.counter+1

        return {'loss': test_loss}

    def on_test_epoch_end(self):
        test_loss = torch.stack(self.test_loss).mean()
        test_acc = torch.stack(self.testAccuracy).mean()
        self.test_loss.clear()
        self.testAccuracy.clear()
        print({"test_loss": test_loss, "testAccuracy": test_acc})
        wandb.log({"test_loss_last":test_loss,"testAccuracy_last":test_acc})






In [ ]:
# function to return key for any value
def get_keyAttn(index_value):
    for character, idx in charToIndLang.items():
        if idx == index_value:
            return character
    return ""


def keyForInput(index):
    for char, idx in char_to_idx_latin.items():
        if idx == index:
            return char
    return ""

def keyForVal(index):
    for char, idx in charToIndLang.items():
        if idx == index:
            return char
    return ""

def get_key_inputAttn(index_value):
    for char, idx in char_to_idx_latin.items():
        if idx == index_value:
            return char
    return ""



In [ ]:
#function will save the input word ,ouput word and predicted word to csv file for attention module
def save_outputs_to_csv(inputs, targets, predictions):
    output_file = 'Output.csv'
    already_present = os.path.isfile(output_file)

    records = {
        'input': inputs,
        'target': targets,
        'predicted': predictions
    }

    dataframe = pd.DataFrame(records)
    dataframe.to_csv(output_file, mode='a', index=False, header=not already_present)





def log_predictions_to_wandb(inputs, targets, predictions):
    table = wandb.Table(columns=["Input", "Target", "Prediction ✅/❌"])

    for inp, tgt, pred in zip(inputs, targets, predictions):
        status_emoji = "✅" if tgt == pred else "❌"
        pred_with_emoji = f"{pred} {status_emoji}"
        table.add_data(inp, tgt, pred_with_emoji)

    wandb.log({"Test Predictions Grid": table})





In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import wandb
from matplotlib.font_manager import FontProperties





def s(input_words, output_words, attentionWeights):
    """Visualise up to 9 attention maps (3×3 grid)."""
    dev_font = FontProperties(fname="/kaggle/input/devnagri/Nirmala.ttf")

    fig, axarr = plt.subplots(3, 3, figsize=(10, 10))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    n_plots = min(len(input_words), len(output_words), len(attentionWeights))

    for idx, ax in enumerate(axarr.ravel()):
        if idx >= n_plots:           # blank panel
            ax.axis("off")
            continue

        # ─────── extract matrix for this pair ───────────────────────────────
        attn = attentionWeights[idx].cpu().detach().numpy()
        attn = attn[1 : len(input_words[idx]) + 1, : len(output_words[idx])]

        # ─────── heat-map ───────────────────────────────────────────────────
        sns.heatmap(
            attn,
            ax=ax,
            cmap="viridis",          # purple → green → yellow
            cbar=False,
            square=True,
            vmin=0,
            vmax=attn.max(),
        )

        # dashed white grid
        ax.set_xticks(np.arange(-0.5, attn.shape[1], 1), minor=True)
        ax.set_yticks(np.arange(-0.5, attn.shape[0], 1), minor=True)
        ax.grid(which="minor", color="white", linestyle="--", linewidth=0.5)
        ax.tick_params(which="minor", bottom=False, left=False)

        # ─────── ticks / labels ─────────────────────────────────────────────
        ax.xaxis.tick_top()  # move x-labels to top
        ax.set_xticks(range(len(output_words[idx])))
        ax.set_xticklabels(
            reversed(output_words[idx]),
            rotation=45,
            ha="right",
            fontproperties=dev_font,
            fontsize=9,
        )

        ax.set_yticks(range(len(input_words[idx])))
        ax.set_yticklabels(
            reversed(input_words[idx]),
            rotation="vertical",
            fontproperties=dev_font,
            fontsize=9,
        )

        ax.set_title(f"Attention {idx + 1}", fontsize=12)

    # log and display
    wandb.log({"Question5_HeatMap": wandb.Image(fig)})
    plt.show()
    plt.close(fig)



#SWEEP CONFIG

In [ ]:
sweep_config = {
    # Bayesian Search for hyperparameters
    "name" : "Bayesian Sweep_Attn",
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "Validation Accuracy"},
    "parameters": {'drop_out': {"values": [0.3,0.5]},
                   'embeddingSize': {"values": [64,128,256]},
                   'hidden_layer_size': {"values": [128,256,512]},
                   'layersEncoder': {"values": [2, 3]},
                   'layersDecoder': {"values": [2, 3]},
                   "cellType": {"values": [ "RNN", "GRU", "LSTM"]},
                   "learningRate": {"values": [1e-3, 0.005]},
                   "bidirectional":{"values":[True, False]},
                   "epochs": {"values": [10, 15]}
                }
}

In [ ]:
def train():
    run = wandb.init(project="DA6401_ASS3_ATTENTION", name="SweepAttention")
    config = run.config

    # Construct a dynamic run name
    wandb.run.name = (
        f"_cell_{config.cellType}_dr_{config.drop_out}"
        f"_em_{config.embeddingSize}_hl_{config.hidden_layer_size}"
        f"_en_{config.layersEncoder}_de_{config.layersDecoder}"
        f"_lr_{config.learningRate}_bi_{config.bidirectional}"
        f"_ep_{config.epochs}"
    )

    # Extract hyperparameters
    hidden_size = config.hidden_layer_size
    embed_dim = config.embeddingSize
    rnn_type = config.cellType
    dec_layers = config.layersDecoder
    enc_layers = config.layersEncoder
    is_bidir = config.bidirectional
    learning_rate = config.learningRate
    dropout = config.drop_out
    num_epochs = config.epochs
    use_attention = True

    input_vocab_size = len(char_to_idx_latin) + 2
    output_vocab_size = len(charToIndLang) + 2

    if not use_attention:
        model = Seq2Seq(
            input_vocab_size,
            output_vocab_size,
            hidden_size,
            embed_dim,
            rnn_type,
            dropout,
            enc_layers,
            dec_layers,
            is_bidir,
            learning_rate
        )
    else:
        model = Seq2SeqAttn(
            input_vocab_size,
            output_vocab_size,
            hidden_size,
            embed_dim,
            rnn_type,
            dropout,
            enc_layers=1,
            dec_layers=1,
            is_bidir=is_bidir,
            lr=learning_rate,
            max_src_len=maxLenEng
        )

    model.to(device)

    trainer = pl.Trainer(
        max_epochs=num_epochs,
        accelerator="gpu",
        devices=1
    )
    trainer.fit(model, train_dataloaders=dataloaderTrain, val_dataloaders=dataloaderVal)


In [ ]:
# # Running Sweeps for with attention
sweep_id = wandb.sweep(sweep_config, project='DA6401_ASS3_ATTENTION')
wandb.agent(sweep_id, train, count = 23)
wandb.finish()

#BEST CONFIG

In [ ]:

epochs=15

hidden_layer_size=256
embeddingSize= 128
cellType="LSTM"
layersDecoder=3
layersEncoder=3
attention=True
bidirectional=True
learningRate=0.001
drop_out=0.5


In [ ]:

model = Seq2SeqAttn(len(char_to_idx_latin)+2, len(charToIndLang)+2, hidden_layer_size, embeddingSize, cellType,drop_out,3,3,bidirectional,learningRate, maxLenEng)

model.to(device)

In [ ]:
wandb.init(project="DA6401_ASS3_ATTENTION",name="SweepAttention")
trainer = pl.Trainer(max_epochs=epochs, accelerator="gpu", devices=1)
trainer.fit(model=model, train_dataloaders=dataloaderTrain, val_dataloaders=dataloaderVal)
trainer.test(model, dataloaderTest)
wandb.finish()